<a href="https://colab.research.google.com/github/himkt/optuna-allennlp/blob/master/notebook/Optuna_AllenNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet "allennlp==v1.1.0rc3" optuna plotly

In [ ]:
!wget https://raw.githubusercontent.com/himkt/optuna-allennlp/master/config/imdb_optuna.jsonnet
!rm -rf result

In [ ]:
import optuna

In [ ]:
def objective(trial: optuna.Trial) -> float:
    trial.suggest_int("embedding_dim", 32, 256)
    trial.suggest_int("max_filter_size", 2, 6)
    trial.suggest_int("num_filters", 32, 256)
    trial.suggest_int("output_dim", 32, 256)
    trial.suggest_float("dropout", 0.0, 0.8)
    trial.suggest_float("lr", 5e-3, 5e-1, log=True)

    executor = optuna.integration.allennlp.AllenNLPExecutor(
        trial=trial,  # trial object
        config_file="./imdb_optuna.jsonnet",  # jsonnet path
        serialization_dir=f"./result/optuna/{trial.number}",  # directory for snapshots and logs
        metrics="best_validation_accuracy"
    )
    return executor.run()

In [ ]:
study = optuna.create_study(
    sampler=optuna.samplers.TPESampler(seed=24),
    study_name="optuna_allennlp",
    direction="maximize",
)

timeout = 60 * 60 * 10  # timeout (sec): 60*60*10 sec => 10 hours
study.optimize(
    objective,
    n_jobs=1,  # number of processes in parallel execution
    n_trials=30,  # number of trials to train a model
    timeout=timeout,  # threshold for executing time (sec)
)

optuna.integration.allennlp.dump_best_config("./imdb_optuna.jsonnet", "best_imdb_optuna.json", study)

In [ ]:
study.trials_dataframe()

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_param_importances(study)